In [1]:
try:
    import pandas
    import mysql.connector
except ImportError:
    !pip install pandas
    !pip install mysql-connector-python
import mysql.connector
import pandas as pd

In [2]:
# Établir la connexion à la base de données
connexion = mysql.connector.connect(
    host="localhost", user="root", password="40363933", database="music_store"
)
if connexion.is_connected():
    print("Connexion réussie à la base de données MySQL")
    # Par exemple :
    curseur = connexion.cursor()
    curseur.execute("SHOW TABLES")
    print("Chargement des données MySQL")
    datas = {
        table[0]: pd.read_sql_query(f"SELECT * FROM {table[0]}", connexion)
        for table in curseur.fetchall()
    }
    # Fermer la connexion une fois que vous avez terminé
    connexion.close()
    print("Connexion à la base de données MySQL fermée")
else:
    print("Échec de la connexion à la base de données MySQL")

Connexion réussie à la base de données MySQL
Chargement des données MySQL
Connexion à la base de données MySQL fermée


C:\Users\yaoko\AppData\Local\Temp\ipykernel_4804\5173280.py:12: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table[0]: pd.read_sql_query(f"SELECT * FROM {table[0]}", connexion)


In [3]:
# 1. Who is the oldest employee (highest level) based on job title?
datas["employees"].sort_values(["levels"], ascending=False).iloc[0, [3, 1, 2]]

title         Senior General Manager
last_name                      Madan
first_name                     Mohan
Name: 8, dtype: object

In [4]:
# 2. Which countries have the highest number of invoices?
datas["invoices"].groupby(["billing_country"]).agg(
    nb_invoices=("invoice_id", "count")
).sort_values("nb_invoices", ascending=False).reset_index()

,billing_country,nb_invoices
0,USA,131
1,Canada,76
2,Brazil,61
3,France,50
4,Germany,41
5,Czech Republic,30
6,Portugal,29
7,United Kingdom,28
8,India,21
9,Chile,13


In [5]:
# 3. What are the top 3 highest values of total invoices?
datas["invoices"].drop_duplicates(["total"]).sort_values("total", ascending=False)[
    "total"
].iloc[:3]

182    23.76
30     19.80
275    18.81
Name: total, dtype: float64

In [6]:
# 4. Which city has the best customers?
# We would like to organize a promotional music festival in the city where we generated the most revenue.
# Write a query that returns a single city having the highest sum of all invoice totals.
# Return both the city name and the sum of all invoice totals.
datas["invoices"].groupby("billing_city").agg(
    invoices_total=("total", "sum")
).sort_values("invoices_total", ascending=False).reset_index().iloc[0, :]

billing_city      Prague
invoices_total    273.24
Name: 0, dtype: object

In [7]:
# 5. Who is the top customer?
# The customer who spent the most money will be declared the top customer.
# Write a query that returns the person who spent the most money.
df = (
    datas["customers"]
    .merge(datas["invoices"], on="customer_id")
    .groupby("customer_id")
    .agg(
        total_spent=("total", "sum"),
        first_name=("first_name", "first"),
        last_name=("last_name", "first"),
    )
    .sort_values("total_spent", ascending=False)
)
df["customer"] = df["first_name"] + " " + df["last_name"]
df.drop(["first_name", "last_name"], axis="columns").iloc[0, :]

total_spent                   144.54
customer       František Wichterlová
Name: 5, dtype: object

In [8]:
# 6. Write a query to return the email, first name, last name of all rock music listeners.
# Return your list ordered alphabetically by email.

df = (
    datas["customers"]
    .merge(datas["invoices"], on="customer_id")
    .merge(datas["invoice_lines"], on="invoice_id")
    .merge(datas["tracks"], on="track_id")
    .merge(datas["genres"].query("name.str.contains('Rock')"), on="genre_id")
    .drop_duplicates(["customer_id"])
    .sort_values("email", ascending=True)
)
df

,customer_id,first_name,last_name,company,address,city,state,country,postal_code,phone,...,quantity,name_x,album_id,media_type_id,genre_id,composer,milliseconds,bytes,unit_price_y,name_y
18,32,Aaron,Mitchell,,696 Osborne Street,Winnipeg,MB,Canada,R3L 2B9,+1 (204) 452-6452,...,1,Love In An Elevator,5,1,1,"Steven Tyler, Joe Perry",321828,10552051,0.99,Rock
90,11,Alexandre,Rocha,Banco do Brasil S.A.,"Av. Paulista, 2022",São Paulo,SP,Brazil,01310-200,+55 (11) 3055-3278,...,1,The Trees,196,1,1,Geddy Lee And Alex Lifeson/Geddy Lee And Neil ...,285126,9345473,0.99,Rock
137,7,Astrid,Gruber,,"Rotenturmstraße 4, 1010 Innere Stadt",Vienne,,Austria,1010,+43 01 5134505,...,1,Who Are You (Single Edit Version),221,1,1,John Entwistle/Pete Townshend,299232,9900469,0.99,Rock
84,4,Bjørn,Hansen,,Ullevålsveien 14,Oslo,,Norway,171,+47 22 44 22 22,...,1,Think About You,90,2,1,,231640,3860275,0.99,Rock
101,39,Camille,Bernard,,"4, Rue Milton",Paris,,France,75009,+33 01 49 70 65 65,...,1,Anthem,196,1,1,Geddy Lee And Alex Lifeson/Geddy Lee And Neil ...,264515,8693343,0.99,Rock
5,8,Daan,Peeters,,Grétrystraat 63,Brussels,,Belgium,1000,+32 02 219 03 03,...,1,Dog Eat Dog,4,1,1,AC/DC,215196,7032162,0.99,Rock
365,56,Diego,Gutiérrez,,307 Macacha Güemes,Buenos Aires,,Argentina,1106,+54 (0)11 4311 4333,...,1,You Got Me Rocking,217,1,1,Jagger/Richards,205766,6734385,0.99,Rock
115,20,Dan,Miller,,541 Del Medio Avenue,Mountain View,CA,USA,94040-111,+1 (650) 644-3358,...,1,La Villa Strangiato,196,1,1,Geddy Lee And Alex Lifeson/Geddy Lee And Neil ...,577488,19137855,0.99,Rock
65,40,Dominique,Lefebvre,,"8, Rue Hanovre",Paris,,France,75002,+33 01 47 42 71 71,...,1,My Hero,82,1,1,"Dave, Taylor, Nate, Chris",260101,8472365,0.99,Rock
61,30,Edward,Francis,,230 Elgin Street,Ottawa,ON,Canada,K2P 1L7,+1 (613) 234-3322,...,1,Whole Lotta Rosie,4,1,1,AC/DC,323761,10547154,0.99,Rock


In [9]:
# 7. Let's invite the artists who have written the most rock music in our dataset.
# Write a query that returns the artist name and total track count of the top 10 rock artists.

df = (
    datas["tracks"]
    .merge(datas["genres"], on="genre_id")
    .merge(datas["albums"], on="album_id")
    .merge(datas["artists"], on="artist_id")
    .merge(datas["genres"].query("name.str.contains('Rock')"), on="genre_id", suffixes=("", "_g"))
    .groupby("name")
    .agg(total_tracks=("track_id", "count"))
    .sort_values("total_tracks", ascending=False)
    .reset_index()
    .iloc[:10, :]
)
df

,name,total_tracks
0,Led Zeppelin,114
1,U2,112
2,Deep Purple,92
3,Iron Maiden,81
4,Pearl Jam,54
5,Van Halen,52
6,Queen,45
7,The Rolling Stones,41
8,Creedence Clearwater Revival,40
9,Kiss,35


In [10]:
# 8. Return all the track names that have a song length longer than the average song length.
# Return the name and milliseconds for each track.
# Order by the song length with the longest songs listed first.

avg_song_length = datas["tracks"]["milliseconds"].mean()
datas["tracks"][datas["tracks"]["milliseconds"] >= avg_song_length].sort_values(
    "milliseconds", ascending=False
)[["name", "milliseconds"]]

,name,milliseconds
2819,Occupation / Precipice,5286953
3223,Through a Looking Glass,5088838
3243,"Greetings from Earth, Pt. 1",2960293
3241,The Man With Nine Lives,2956998
3226,"Battlestar Galactica, Pt. 2",2956081
...,...,...
1386,22 Acacia Avenue,395572
1863,The Unforgiven II,395520
1896,The Shortest Straw,395389
3412,"Concerto for Clarinet in A Major, K. 622: II. ...",394482


In [11]:
# 9. Find how much amount spent by each customer on artists?
# Write a query to return customer name, artist name, and total spent.
df = datas["invoice_lines"].copy()
df["price_total"] = df["unit_price"] * df["quantity"]
df = (
    df.merge(datas["invoices"], on="invoice_id")
    .merge(datas["customers"], on="customer_id")
    .merge(datas["tracks"], on="track_id")
    .merge(datas["albums"], on="album_id")
    .merge(datas["artists"], on="artist_id", suffixes=("", "_a"))
    .groupby(["customer_id", "artist_id"])
    .agg(
        first_name=("first_name", "first"),
        last_name=("last_name", "first"),
        artist_name=("name_a", "first"),
        total_spent=("price_total", "sum"),
    )
)
df.reset_index(drop=True)

,first_name,last_name,artist_name,total_spent
0,Luís,Gonçalves,AC/DC,7.92
1,Luís,Gonçalves,Aerosmith,2.97
2,Luís,Gonçalves,Alanis Morissette,1.98
3,Luís,Gonçalves,Apocalyptica,0.99
4,Luís,Gonçalves,Audioslave,1.98
...,...,...,...,...
2184,Rishabh,Mishra,Van Halen,0.99
2185,Rishabh,Mishra,Scorpions,0.99
2186,Rishabh,Mishra,Cake,0.99
2187,Rishabh,Mishra,Christopher O'Riley,0.99


In [12]:
# 10. We want to find out the most popular music genre for each country.
# We define the most popular genre as the genre with the highest number of track purchases.
# Write a query that returns each country along with the top genre.
# For countries where the maximum number of purchases is shared, return all genres.
df = datas["invoice_lines"].copy()
df["price_total"] = df["unit_price"] * df["quantity"]
df = (
    datas["invoice_lines"]
    .merge(datas["invoices"], on="invoice_id")
    .merge(datas["tracks"], on="track_id")
    .merge(datas["genres"], on="genre_id", suffixes=("", "_g"))
    .groupby(["billing_country", "genre_id"])
    .agg(
        country=("billing_country", "first"),
        genre_name=("name_g", "first"),
        nb_purchases=("track_id", "count"),
    )
)
df.reset_index(drop=True).groupby("country").agg(
    max_nb_purchases=("nb_purchases", "max")
).reset_index().merge(df, on="country").query("nb_purchases == max_nb_purchases").drop(
    ["max_nb_purchases"], axis="columns"
)

,country,genre_name,nb_purchases
2,Argentina,Alternative & Punk,17
10,Australia,Rock,34
21,Austria,Rock,40
29,Belgium,Rock,26
39,Brazil,Rock,205
52,Canada,Rock,333
67,Chile,Rock,61
78,Czech Republic,Rock,143
92,Denmark,Rock,24
98,Finland,Rock,46
